# SPRINT - 2

## Item - Levantamento dos cálculos dos indicadores.

## Item - Levantamento do procedimento de consolidação dos dados de produção e apontamentos.

## Item - Desenvolvimento do processamento de Limpeza, Formatação e Classificação dos dados, e gravação na "Refined Data".

<br>
<br>

Autor.: Sérgio C. Medina

#### Declaração dos Pacotes, Libs ou Classes utilizadas no processo.

In [1]:
# Declaração dos Pacotes, Libs ou Classes utilizadas no processo.
import os
import io
import math
import pandas as pd
import gcsfs
import pyarrow
import pyarrow.parquet as pq
from google.cloud import storage
from datetime import datetime, timedelta



# configurando variavel de ambiente com o arquivo de credenciais para conexão GCP
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "..\..\secrets\edc-igti-smedina-4920e12ac565.json"

# Conexão com o Cloud Storage e criação do objeto bucket referente a área de dados
storage_client = storage.Client()
bucket = storage_client.get_bucket("edc-pa-i4-data")

In [2]:
# Função para leitura do arquivo CSV no GCS convertando para Pandas Dataframe
def gcp_csv_to_df(folder, dtexec, lineprod=None, sep=";"):

    # formata o padrão do nome do arquivo
    file_name = None
    if (dtexec is not None and lineprod is None):
        file_name = f"{dtexec}.csv"
    elif (dtexec is not None and lineprod is not None):
        file_name = f"{lineprod}-{dtexec}.csv"
    else:
        raise Exception("Invalid parameters.")

    df = None

    try:
        # define o objeto blob com o path para o arquivo no GCS
        blob = bucket.blob(f"raw-data-zone/{folder}/{file_name}")

        # Faz o download no formato em bytes e posiciona no index 0
        byte_stream = io.BytesIO()
        blob.download_to_file(byte_stream)
        byte_stream.seek(0)

        # Converte para o Dataframe em Pandas
        df = pd.read_csv(byte_stream, sep=sep)
    except Exception as e:
        print("Exception:", e)

    return df


# Função para realizar o delete do blob no GCS
def delete_blob(path_name):
    if path_name is not None:
        print('SEARCHING TO DELETE:', path_name)
        blobs = bucket.list_blobs(prefix=path_name)
        for blob in blobs:
            if blob.exists():
                print('DELETE:',blob.path)
                blob.delete()
    else:
        print('ERROR: path_name is None!')



In [3]:
# Função para calcular data turno com base em uma data e hora
def calc_dtprod(dt):

    # Se dt for string
    if isinstance(dt,str):
        dt = datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")

    # separa a data e hora
    my_date = dt.date()
    my_hora = dt.time()

    # Formato hora 
    FMT = '%H:%M:%S'

    # Verifica periodo entre 06:00:00 e 23:59:59
    start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
    end_time = datetime.strptime('23:59:59', '%H:%M:%S').time()

    if (my_hora >= start_time and my_hora <= end_time):
        return my_date
    elif (my_hora < start_time):
        prevDay = my_date + timedelta(days=-1)
        return prevDay
    else:
        return None

# Função para calcular o ID do TURNO baseado em uma data e hora
def calc_idturno(dt):

    # Se dt for string
    if isinstance(dt,str):
        dt = datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")

    # separa a hora
    my_hora = dt.time()

    id = None

    if is_between(my_hora,   '06:00:00', '13:59:59'):
        id = 1
    elif is_between(my_hora, '14:00:00', '21:59:59'):
        id = 2
    elif is_between(my_hora, '22:00:00', '23:59:59'):
        id = 3
    elif is_between(my_hora, '00:00:00', '05:59:59'):
        id = 3

    return id


# Funcão que retorno a data e hora de final do turno
def end_time(dtprd, idturno):

    # Se dtprd for string
    if isinstance(dtprd,str):
        dtprd = datetime.strptime(dtprd, "%Y-%m-%d").date()

    if idturno == 1:
        dt = f'{datetime.strftime(dtprd, "%Y-%m-%d")} 13:59:59'
    elif idturno == 2:
        dt = f'{datetime.strftime(dtprd, "%Y-%m-%d")} 21:59:59'
    elif idturno == 3:
        dtprd = dtprd + timedelta(days=1)
        dt = f'{datetime.strftime(dtprd, "%Y-%m-%d")} 05:59:59'

    return dt


# Função que verifica se uma data está dentro do período definido
def is_between(check_time, start_time, end_time, format_str_time='%H:%M:%S'):

    if isinstance(check_time,str):
        check_time = datetime.strptime(check_time, format_str_time).time()
    
    if isinstance(start_time,str):
        start_time = datetime.strptime(start_time, format_str_time).time()
    
    if isinstance(end_time,str):
        end_time = datetime.strptime(end_time, format_str_time).time()

    return (start_time <= check_time <= end_time)



# Compara se a data inicio e final informada estão na mesma data de produção
# se sim retorna a data de produção, senão retorna None
def compate_dtprod(di, df):
    di = calc_dtprod(di)
    df = calc_dtprod(df)

    if (di == df):
        return di
    else:
        return None
  
# defining a function for round down.
def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

#### Dados na "raw-data-zone" pasta "DATAOP"

In [ ]:
dtexec = "2021-11-08"
folder="dataop"

df_dataop = gcp_csv_to_df(folder=folder, dtexec=dtexec, sep=";")

if df_dataop is not None:
    # Limpa os dados nulos
    df_dataop.dropna(inplace=True)

    # Define os tipos de cada coluna
    df_dataop['OP'] = df_dataop['OP'].astype(str)
    df_dataop['CODMAT'] = df_dataop['CODMAT'].astype(str)
    df_dataop['LOTEFAB'] = df_dataop['LOTEFAB'].astype(str)
    df_dataop['DTINI'] = pd.to_datetime(df_dataop['DTINI'])
    df_dataop['DTFIM'] = pd.to_datetime(df_dataop['DTFIM'])
    df_dataop['QTDPLAN'] = df_dataop['QTDPLAN'].astype(int)

    # Definindo a Data de Produção DTPROD
    df_dataop.insert(0, 'DTPROD', None)
    df_dataop['DTPROD'] = df_dataop.apply(lambda row:compate_dtprod(row['DTINI'], row['DTFIM']),axis=1)
    df_dataop['DTPROD'] = df_dataop['DTPROD'].astype(str)

    # save parquet - gcp
    # https://gist.github.com/lpillmann/fa1874c7deb8434ca8cba8e5a045dde2
    # https://blog.datasyndrome.com/python-and-parquet-performance-e71da65269ce

    # Verifica se existe arquivos no path para deletar
    gs_blobs=f"processing-zone/{folder}/DTPROD={dtexec}"
    delete_blob(path_name=gs_blobs)

    # Gravando na Processing-Zone
    gs_path=f"edc-pa-i4-data/processing-zone/{folder}"
    gs = gcsfs.GCSFileSystem()

    table = pyarrow.Table.from_pandas(df_dataop)
    pq.write_to_dataset(
        table,
        gs_path,
        partition_cols=['DTPROD'],
        filesystem=gs,
        compression='snappy',
        flavor='spark'
    )

    print("SUCCESS:", gs_path)
      

In [ ]:
df_dataop.dtypes

In [ ]:
df_dataop

# Somarizando os dados Nuloes
#df_dataop.isnull().sum()

# Limpando os dados Nulos
#df_dataop.dropna(inplace=True)

In [ ]:
dataset = pq.ParquetDataset(
    f"edc-pa-i4-data/processing-zone/{folder}", 
    filesystem=gs
#    filters=[('DTPROD', '=', '2021-11-08')]
#    use_threads=True
)
df = dataset.read_pandas().to_pandas()

In [ ]:
df

#### Dados na "raw-data-zone" pasta "DATAPROD"

In [4]:
dtexec = "2021-11-08"
lineid = "101"

# Carregando os dados do arquivo da data e linha informados
df_dataprod = gcp_csv_to_df(folder="dataprod", dtexec=dtexec, lineprod=lineid, sep=";")

if df_dataprod is not None:

    # verificando se existe dados do dia posterior para completar o turno 3
    nextDay = datetime.strptime(dtexec, "%Y-%m-%d") + timedelta(days=1)
    print("dtexec:", dtexec, " next:", nextDay)    
    df = gcp_csv_to_df(folder="dataprod", dtexec=nextDay.strftime("%Y-%m-%d"), lineprod=lineid, sep=";")
    if df is not None:
        df_dataprod = df_dataprod.append(df, sort=False, ignore_index=True)

    # Limpa os dados nulos
    df_dataprod.dropna(inplace=True)

    # Alterando o delimitador decimal de "," para "."
    df_dataprod['TOTMIN']=df_dataprod.TOTMIN.str.replace(',','.')

    # Define os tipos de cada coluna
    df_dataprod['OP'] = df_dataprod['OP'].astype(str)
    df_dataprod['LINE'] = df_dataprod['LINE'].astype(int)
    df_dataprod['LINE'] = df_dataprod['LINE'].astype(str)
    df_dataprod['TIMESTAMP'] = pd.to_datetime(df_dataprod['TIMESTAMP'])
    df_dataprod['BATCH'] = df_dataprod['BATCH'].astype(str)
    df_dataprod['TIMER'] = df_dataprod['TIMER'].astype(str)
    df_dataprod['TOTMIN'] = df_dataprod['TOTMIN'].astype(float)
    df_dataprod['STSID'] = df_dataprod['STSID'].astype(int)
    df_dataprod['STSDS'] = df_dataprod['STSDS'].astype(str)
    df_dataprod['PC'] = df_dataprod['PC'].astype(float)
    df_dataprod['GOOD'] = df_dataprod['GOOD'].astype(float)
    df_dataprod['REJECT'] = df_dataprod['REJECT'].astype(float)

    # Definindo a Data de Produção DTPROD
    df_dataprod.insert(0, 'DTPROD', None)
    df_dataprod['DTPROD'] = df_dataprod.apply(lambda row:calc_dtprod(row['TIMESTAMP']),axis=1)
    df_dataprod['DTPROD'] = df_dataprod['DTPROD'].astype(str)
    
    # Definindo a ID do Turno de Produção IDTURNO
    df_dataprod.insert(1, 'IDTURNO', 0)
    df_dataprod['IDTURNO'] = df_dataprod.apply(lambda row:calc_idturno(row['TIMESTAMP']),axis=1)
    df_dataprod['IDTURNO'] = df_dataprod['IDTURNO'].astype(int)

    # Seleciona apenas o periodo referente ao DTPROD
    df_dataprod = df_dataprod.loc[ df_dataprod['DTPROD'] == dtexec ]

    # reindex
    df_dataprod = df_dataprod.reset_index(drop=True)




dtexec: 2021-11-08  next: 2021-11-09 00:00:00


In [ ]:
df_dataprod.dtypes

In [5]:
#df.groupby(['col1', 'col2']).agg({'col3':'sum','col4':'sum'})

df = pd.DataFrame(df_dataprod.groupby(
    ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH']
).agg(
    {
        'TOTMIN': 'sum',
        'PC': 'sum',
        'GOOD': 'sum',
        'REJECT': 'sum',
        'TIMESTAMP': 'max'
    }
)).reset_index()


df['TIMELIMIT'] = df.apply(lambda row:end_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TIMELIMIT'] = pd.to_datetime(df['TIMELIMIT'])
df['TIMEDIFFMIN'] = (df['TIMELIMIT']-df['TIMESTAMP']).dt.seconds/60
df['WIP'] = df.apply(lambda row:round_down(row['TIMEDIFFMIN'], 0),axis=1)

df.drop(['TIMESTAMP', 'TIMELIMIT', 'TIMEDIFFMIN'], axis=1, inplace=True)



In [6]:
df

,DTPROD,IDTURNO,LINE,OP,BATCH,TOTMIN,PC,GOOD,REJECT,WIP
0,2021-11-08,1,101,211108L101,TB70394,474.87,444.0,394.0,50.0,5.0
1,2021-11-08,2,101,211108L101,TB70394,479.83,452.0,450.0,2.0,6.0
2,2021-11-08,3,101,211108L101,TB70394,485.93,456.0,453.0,3.0,0.0


In [7]:
df.TOTMIN.sum()

1440.63

In [8]:
df_dataprod.TOTMIN.sum()

1440.63